In [1]:
import ipycytoscape
import networkx as nx
from itertools import combinations, groupby
from random import SystemRandom, random, choice, sample
from IPython.display import display
from Node import Node
from Edge import Edge
from Graph import Graph
from Flight import Flight
from math import ceil
import json

In [2]:
from Node import Node
from Edge import Edge


class Graph:

    graph: dict = {}

    @staticmethod
    def create_graph():
        Graph.graph = {i: {} for i in Node.nodes.keys()}
        for egde in Edge.egdes:
            Graph.graph[egde.start][egde.end] = egde.weight
            Graph.graph[egde.end][egde.start] = egde.weight

    def __repr__(self):
        return f"Graph({Graph.graph})"

In [3]:
import Flight


class Node:

    nodes: dict = {}
    nodes_quantity: int = 0

    def __init__(self, id: int, latitude: float, longitude: float):
        self.id: int = id
        self.latitude: float = latitude
        self.longitude: float = longitude
        self.flights: list[Flight] = []
        Node.nodes[id] = self
        Node.nodes_quantity += 1

    def get_flights(self):
        if self.flights:
            return self.flights
        else:
            return None

    def add_flight(self, flight: Flight):
        self.flights.append(flight)

    def __repr__(self):
        return f"Node(id={self.id}, latitude={self.latitude}, longitude={self.longitude})"


In [4]:
import networkx as nx
from itertools import combinations, groupby
from random import SystemRandom, random, choice, sample, randint
from Node import Node
from Edge import Edge
from Graph import Graph
from Flight import Flight
from math import ceil
import json
import matplotlib.pyplot as plt


def random_world_path_generator(n, p):
    edges = combinations(range(n), 2)
    G = nx.Graph()
    G.add_nodes_from(range(n))
    if p <= 0:
        return G
    if p >= 1:
        return nx.complete_graph(n, create_using=G)
    for _, node_edges in groupby(edges, key=lambda x: x[0]):
        node_edges = list(node_edges)
        random_edge = choice(node_edges)
        G.add_edge(*random_edge)
        for e in node_edges:
            if random() < p:
                G.add_edge(*e)

    for (start, end) in G.edges:
        G.edges[start, end]['weight'] = randint(0, 20)
    return G


def create_world_path(nodes_quantity):
    random_graph = random_world_path_generator(nodes_quantity, 0.1)

    for node in random_graph.nodes:
        Node(node, SystemRandom().uniform(-90, 90), SystemRandom().uniform(-180, 180))

    for start, end, weight in random_graph.edges.data('weight'):
        Edge(start, end, weight)

    Graph.create_graph()
    
    return random_graph


In [5]:
G = random_world_path_generator(20,0.1)
cyto = ipycytoscape.CytoscapeWidget()
cyto.graph.add_graph_from_networkx(G)

cyto.set_style(
    [
        {
            'selector': 'node',
            'style': {
                'label': 'data(id)'
            }
        },
        {
            'selector': 'edge',
            'style': {
                'label': 'data(weight)'
            }
        }
    ]
)

display(cyto)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

In [6]:
from math import inf
import heapq

n = 4


G = create_world_path(n)
print(G)
cyto = ipycytoscape.CytoscapeWidget()
cyto.graph.add_graph_from_networkx(G)

cyto.set_style(
    [
        {
            'selector': 'node',
            'style': {
                'label': 'data(id)'
            }
        },
        {
            'selector': 'edge',
            'style': {
                'label': 'data(weight)'
            }
        }
    ]
)

display(cyto)

print(Node.nodes_quantity)

origin_node = 5

graphSPD: dict = dict([(i, inf) for i in range(Node.nodes_quantity)])
graphSPD[origin_node] = 0


def dijkstra(id_origin):
    global graphSPD
    queue: list = []
    heapq.heappush(queue, (0, id_origin))  # (SPD, node)
    while len(queue) > 0:
        spdU, u = heapq.heappop(queue)
        for v, d in Graph.graph[u].items():
            if spdU + d < graphSPD[v]:
                graphSPD[v] = spdU + d
                heapq.heappush(queue, (graphSPD[v], v))


dijkstra(origin_node)
print(graphSPD)

Graph with 4 nodes and 3 edges


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

4


KeyError: 5